In [4]:
import os
import shutil
import rasterio
from glob import glob

In [5]:
data = os.getcwd() + "\\data"
months_list = glob(data+"\\*_archive")
months_list

['C:\\Users\\Hello\\Documents\\remote-sensing\\APIs\\SentinelSat\\data\\Godavari2016_archive',
 'C:\\Users\\Hello\\Documents\\remote-sensing\\APIs\\SentinelSat\\data\\Godavari2017_archive',
 'C:\\Users\\Hello\\Documents\\remote-sensing\\APIs\\SentinelSat\\data\\Godavari2018_archive',
 'C:\\Users\\Hello\\Documents\\remote-sensing\\APIs\\SentinelSat\\data\\Krishna2016_archive',
 'C:\\Users\\Hello\\Documents\\remote-sensing\\APIs\\SentinelSat\\data\\Krishna2017_archive',
 'C:\\Users\\Hello\\Documents\\remote-sensing\\APIs\\SentinelSat\\data\\Krishna2018_archive']

In [6]:
if not os.path.exists(os.getcwd()+"\\stacked_images"):
    stack_folder = os.makedirs(os.getcwd()+"\\stacked_images")
for month in months_list:
    dst = month.split('\\')[-1]
    if not os.path.exists(os.getcwd()+"\\stacked_images\\"+dst):
        os.makedirs(os.getcwd()+"\\stacked_images\\"+dst)

In [8]:
#stacking
for month in months_list:
    print(month)
    data_list = glob(month+"\\*")
    num_scenes = len(data_list)
    i = 0
    for scene in data_list:
        i = i + 1
        #path  =  scene + "\\GRANULE"
        data_folder = glob(scene + "\\*")
        #data_folder[0] =  data_folder[0] + "\\IMG_DATA\\R10m"
        tifFiles = []
        tifFiles.append(glob(data_folder[0] + "\\B08.tif")[0])
        tifFiles.append(glob(data_folder[0] + "\\B04.tif")[0])
        tifFiles.append(glob(data_folder[0] + "\\B03.tif")[0])
        tifFiles.append(glob(data_folder[0] + "\\B02.tif")[0])
        with rasterio.open(tifFiles[0]) as src0:
            meta = src0.meta
        meta.update(count = len(tifFiles))
        dest = scene.split('\\')[-2].split('_')[0] + scene.split('\\')[-1][12:19]
        dest_name = dest + ".tif"
        dest = data_folder[0] +"\\"+ dest_name
        if not os.path.exists(dest):
            with rasterio.open(dest, 'w', **meta) as dst:
                for id, layer in enumerate(tifFiles, start=1):
                    with rasterio.open(layer) as src1:
                        dst.write_band(id, src1.read(1))
        print("(" + str(i)  + "/" + str(num_scenes) + ") Stacked " + dest_name + "...")
        stack_dest_name = scene.split("\\")[-1].split('.')[0] + ".tif"
        stack_dest = os.getcwd() + "\\stacked_images\\"+ month.split('\\')[-1] + "\\" + stack_dest_name
        if not os.path.exists(stack_dest):
            shutil.copy2(dest, stack_dest)
        print("(" + str(i)  + "/" + str(num_scenes) + ") Copied to stacked_data ...")

C:\Users\Hello\Documents\remote-sensing\APIs\SentinelSat\data\Godavari2016_archive
C:\Users\Hello\Documents\remote-sensing\APIs\SentinelSat\data\Godavari2017_archive
C:\Users\Hello\Documents\remote-sensing\APIs\SentinelSat\data\Godavari2018_archive
(1/3) Stacked Godavari20180180225.tif...
(1/3) Copied to stacked_data ...
(2/3) Stacked Godavari20180180225.tif...
(2/3) Copied to stacked_data ...
(3/3) Stacked Godavari20180180225.tif...
(3/3) Copied to stacked_data ...
C:\Users\Hello\Documents\remote-sensing\APIs\SentinelSat\data\Krishna2016_archive
C:\Users\Hello\Documents\remote-sensing\APIs\SentinelSat\data\Krishna2017_archive
(1/4) Stacked Krishna20170170210.tif...
(1/4) Copied to stacked_data ...
(2/4) Stacked Krishna20170170210.tif...
(2/4) Copied to stacked_data ...
(3/4) Stacked Krishna20170170210.tif...
(3/4) Copied to stacked_data ...
(4/4) Stacked Krishna20170170210.tif...
(4/4) Copied to stacked_data ...
C:\Users\Hello\Documents\remote-sensing\APIs\SentinelSat\data\Krishna2018

In [ ]:
#cleanup data folder

for month in months_list:
    print(month)
    data_list = glob(month+"\\*")
    for scene in data_list:
        shutil.rmtree(scene)
        print("Deleted scene " + scene.split('\\')[-1].split('.')[0] + "...")